### cGAN

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets
import torchvision.transforms as transforms
from torchvision.utils import save_image
from torch.utils.data import DataLoader
import argparse
import os
import numpy as np
import math

In [2]:
seed = 0
torch_seed = torch.manual_seed(seed)

parser =  argparse.ArgumentParser()
parser.add_argument('--n_epoch', type=int, default=200)
parser.add_argument('--batch_size', type=int, default=50)
parser.add_argument('--lr', type=float, default=0.0001)
parser.add_argument('--b1', type=float, default=0.5)
parser.add_argument('--b2', type=float, default=0.999)
parser.add_argument('--n_cpu', type=int, default=8)
parser.add_argument('--latent_dim', type=int, default=100)
parser.add_argument('--img_size', type=int, default=28)
parser.add_argument('--channels', type=int, default=1)
parser.add_argument('--sample_interval', type=int, default=1000)
parser.add_argument('--condition_size', type=int, default=10)
args = parser.parse_args('')

print(args)


Namespace(n_epoch=200, batch_size=50, lr=0.0001, b1=0.5, b2=0.999, n_cpu=8, latent_dim=100, img_size=28, channels=1, sample_interval=400, condition_size=10)


In [3]:
img_shape = (args.channels, args.img_size, args.img_size)
cuda = True if torch.cuda.is_available() else False

In [4]:
class Generator(nn.Module):
  def __init__(self):
    super(Generator, self).__init__()

    def block(input_dim, output_dim, normalize=True):
      layers = [nn.Linear(input_dim, output_dim)]
      if normalize:
        layers.append(nn.BatchNorm1d(output_dim, 0.8))
      layers.append(nn.LeakyReLU(0.2, inplace=True))
      return layers
    
    self.model = nn.Sequential(
      *block(args.latent_dim + args.condition_size, 128, normalize=False),
      *block(128, 256),
      *block(256, 512),
      *block(512, 1024),
      nn.Linear(1024, int(np.prod(img_shape))),
      nn.Tanh()
    )
  
  def forward(self, noise, labels):
    gen_input = torch.cat((self.))
    img = self.model(z)
    img = img.view(img.size(0), *img_shape)
    return img


In [5]:
class Discriminator(nn.Module):
  def __init__(self):
    super(Discriminator, self).__init__()

    self.model = nn.Sequential(
      nn.Linear(int(np.prod(img_shape)), 512),
      nn.LeakyReLU(0.2, inplace=True),
      nn.Linear(512, 256),
      nn.LeakyReLU(0.2, inplace=True),
      nn.Linear(256, 1),
      nn.Sigmoid()
    )
  
  def forward(self, img):
    img_flat = img.view(img.size(0), -1)
    validity = self.model(img_flat)

    return validity

In [6]:
# loss function
adversarial_loss = torch.nn.BCELoss() # Binary Coss Entropy

# init generator & discriminator
generator = Generator()
discriminator = Discriminator()

if cuda:
  generator.cuda()
  discriminator.cuda()
  adversarial_loss.cuda()

In [7]:
# configure data loader
os.makedirs('../data/mnist', exist_ok=True)
dataloader = DataLoader(
  datasets.MNIST(
    '../data/mnist',
    train=True,
    download=True,
    transform=transforms.Compose(
      [transforms.Resize(args.img_size), transforms.ToTensor(), transforms.Normalize([0.5], [0.5])]
    )
  ),
  batch_size=args.batch_size,
  shuffle=True
)

In [8]:
# optimizer
optimizer_G = torch.optim.Adam(generator.parameters(), lr=args.lr, betas=(args.b1, args.b2))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=args.lr, betas=(args.b1, args.b2))

# 텐서 변환 함수
Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor

In [9]:
os.makedirs("cGAN_img", exist_ok=True)

In [ ]:
for epoch in range(args.n_epochs):
  for i, (imgs, labels) in enumerate(dataloader):

    # Adversarial groun truths
    # 진짜 이미지와 가짜 이미지에 대한 정답 레이블 생성
    real = Tensor(imgs.size(0), 1).fill_(1.0)
    fake = Tensor(imgs.size(0), 1).fill_(0.0)

    # configure input
    real_imgs = imgs.type(Tensor)

    # label encoded
    label_encoded = F.one_hot(labels, num_classes=args.condition_size)

    if cuda():
      label_encoded.cuda()
    
    real_concat = torch.cat((real_imgs, label_encoded), 1)

    # train generator
    optimizer_G.zero_grad()
    
    # noise 생성
    z = Tensor(np.random.normal(0, 1, (imgs.shape[0], args.latent_dim)))

    # generator로 이미지 생성
    gen_imgs = generator(z)

    # C(D(G(z), real))
    g_loss = adversarial_loss(discriminator(gen_imgs), real)
    
    g_loss.backward()
    optimizer_G.step()
    
    # train discriminator

    optimizer_D.zero_grad()
    
    real_loss = adversarial_loss(discriminator(real_imgs), real)
    fake_loss = adversarial_loss(discriminator(gen_imgs.detach()), fake)
    d_loss = (real_loss + fake_loss) / 2

    d_loss.backward()
    optimizer_D.step()

    batches_done = epoch * len(dataloader) + i
    if batches_done % args.sample_interval == 0:
      save_image(gen_imgs.data[:25], f'GAN_img/{batches_done}.png', nrow=5, normalize=True)
      
  print(f'[EPOCH {epoch}/{args.n_epochs}] [D loss: {d_loss.item():.2f}] [G loss: {g_loss.item():.2f}]')

In [44]:
a = nn.Embedding(10, 10)
a(torch.arange(3))

tensor([[ 0.4407, -0.4843,  0.9224, -0.1106, -0.6360, -0.6821,  0.2184,  1.1475,
         -0.2259, -0.0196],
        [-0.3078, -0.4204, -1.0775, -0.7831, -0.6350, -0.3287, -0.4010,  0.4889,
         -2.0756,  1.4061],
        [ 1.1139,  0.1989, -0.4361, -1.0456, -1.4605,  0.1574, -0.1377, -1.0093,
         -0.0158, -1.7044]], grad_fn=<EmbeddingBackward0>)

In [45]:
np.random.randint(0, 10, 10)

array([2, 8, 1, 3, 8, 7, 8, 7, 0, 3])